# 雀魂網頁相關(?)
處理 python bot -> 網頁雀魂的各種雜事...

(因為這不太重要+我懶所以)
座標目前都用我本機量的絕對座標，理論上不同電腦會有偏差
如果要量的話可以用下面這段程式碼抓 x,y 座標(但挺麻煩qwq)

In [1]:
'''
from pynput import mouse, keyboard

def on_click(x, y, button, pressed):
    if pressed:  # 滑鼠點擊時打印座標
        print(f"滑鼠點擊座標：x={x}, y={y}")

def on_press(key):
    try:
        # 如果按下的鍵是 'e'，退出程式
        if key.char == 'e':
            print("偵測到按下 'e' 鍵，結束程式。")
            return False  # 停止鍵盤監聽
    except AttributeError:
        # 忽略非字母按鍵
        pass

# 啟動滑鼠監聽器
with mouse.Listener(on_click=on_click) as mouse_listener, keyboard.Listener(on_press=on_press) as keyboard_listener:
    print("請點擊滑鼠查看座標，按下 'e' 鍵結束程式。")
    keyboard_listener.join()  # 等待鍵盤監聽器退出
    mouse_listener.stop()  # 停止滑鼠監聽器

'''

'\nfrom pynput import mouse, keyboard\n\ndef on_click(x, y, button, pressed):\n    if pressed:  # 滑鼠點擊時打印座標\n        print(f"滑鼠點擊座標：x={x}, y={y}")\n\ndef on_press(key):\n    try:\n        # 如果按下的鍵是 \'e\'，退出程式\n        if key.char == \'e\':\n            print("偵測到按下 \'e\' 鍵，結束程式。")\n            return False  # 停止鍵盤監聽\n    except AttributeError:\n        # 忽略非字母按鍵\n        pass\n\n# 啟動滑鼠監聽器\nwith mouse.Listener(on_click=on_click) as mouse_listener, keyboard.Listener(on_press=on_press) as keyboard_listener:\n    print("請點擊滑鼠查看座標，按下 \'e\' 鍵結束程式。")\n    keyboard_listener.join()  # 等待鍵盤監聽器退出\n    mouse_listener.stop()  # 停止滑鼠監聽器\n\n'

### Import & Parameters

In [2]:
from selenium import webdriver
from pynput import mouse, keyboard
from selenium.webdriver.common.action_chains import ActionChains
import pyautogui
import time
import sys
from enum import Enum, auto

class State(Enum):
    Loading = auto()
    Home = auto()
    InGame = auto()


OP_INTERVAL = 1 # click interval
current_state = None
current_hand = None
driver = None
main_init = False
auto_running = False

### Start Game & Auto Login

開啟一個 Chrome 瀏覽器連到雀魂並登入測試帳號

In [3]:
def start_game():
    global current_state, driver
    current_state = State.Loading
    driver = webdriver.Chrome()
    driver.get("https://game.maj-soul.com")
    driver.maximize_window()
    time.sleep(8)

    pyautogui.moveTo(1200, 450)
    pyautogui.click()
    pyautogui.write("paopaodmm@gmail.com", interval=0.05)
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1200, 520)
    pyautogui.click()
    pyautogui.write("paopao123", interval=0.05)
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1200, 670)
    pyautogui.click()

    # wait for the page to load
    time.sleep(12)

    # close the pop-up
    pyautogui.moveTo(1640, 370)
    pyautogui.click()
    time.sleep(OP_INTERVAL)
    current_state = State.Home
    print("Game started")

### Close browser

In [4]:
def close_browser():
    global driver
    driver.quit()
    driver = None
    print("Browser closed.")

### 自動創機器人房間

必須在首頁才能呼叫

In [5]:
def enter_bot_game():
    global current_state
    if current_state != State.Home:
        print("Not in home state, Abort")
        return
    # create bot room
    pyautogui.moveTo(1280, 750)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1280, 500)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(965, 920)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # add 3 bots
    pyautogui.moveTo(700, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1000, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1300, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # start the game
    pyautogui.moveTo(1080, 950)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # wait for the game to start
    time.sleep(15)
    current_state = State.InGame
    print("Bot game started")

### Leave game

In [6]:
def leave_bot_game():
    global current_state
    if current_state != State.InGame:
        print("Not in game state, Abort")
        return
    
    pyautogui.moveTo(1645, 235)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(825, 710)
    pyautogui.click()
    time.sleep(OP_INTERVAL)
    
    # Return to the main page time
    time.sleep(5)
    current_state = State.Home
    print("Bot game left")

### 嵌入 js code
github上的code ><

In [7]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



def inject_js():
    global driver
    
    # with open("AlphaJong_1.3.1.user.js", "r", encoding="utf-8") as js_file:
    #     js_code = js_file.read()
    with open("web.js", "r", encoding="utf-8") as js_file:
        js_code = js_file.read()
    
    driver.execute_script(js_code)
    print("JavaScript 已成功嵌入！")
    
    # # # 等待資源載入完成
    # # WebDriverWait(driver, 10).until(
    # #     lambda d: d.execute_script("return typeof toggleRun === 'function';")
    # # )

    # # # 呼叫函數
    # # driver.execute_script("toggleRun();")

    # # # 呼叫 JavaScript 函數
    # # driver.execute_script("toggleRun();")  # 啟動機器人
    # print("已啟動機器人。")

In [8]:
def filter_hand_data(data):
    filtered_data = {'dora': data['dora'], 'type': data['type'], 'index': data['index']}
    return filtered_data
def update_hand_data(new_data):
    global current_hand
    filtered_data = [filter_hand_data(tile) for tile in new_data]
    current_hand = filtered_data

def visualize_tile(tile):
    dora, tile_type, index = tile['dora'], tile['type'], tile['index']
    if tile_type == 0:
        tile_type = '筒'
    elif tile_type == 1:
        tile_type = '萬'
    elif tile_type == 2:
        tile_type = '條'
    else:
        tile_type = '字'
    return f"{index} {tile_type} {'Dora' if dora else ''}"

def print_current_hand():
    global current_hand
    for i, tile in enumerate(current_hand):
        print(f"tile {i}: {visualize_tile(tile)}")

## Test

這邊可以加入更多測試項目

像是按下 's' 開啟一場 bot game，按下 'e' 跳回首頁等等

In [14]:

def on_press(key):
    global auto_running, main_init, driver
    hand_data = None
    try:
        # 基本上執行完之前都會被 block 住
        if key.char == 'e':
            print("'e' key pressed, leaving bot game.")
            leave_bot_game()
        elif key.char == 's':
            print("'s' key pressed, starting bot game.")
            enter_bot_game()
        elif key.char == 'q':
            print("'q' key pressed, closing browser.")
            close_browser()
            sys.exit(0)
        elif key.char == 'h':
            print("'h' key pressed, Helloworldh.")
            driver.execute_script("helloword();")
        elif key.char == 'g':
            print("'g' key pressed, get Hand Data.")
            hand_data = driver.execute_script("return GetHandData();")
            if hand_data is None:
                print("手牌資料未返回，請檢查 JavaScript 函數的返回值！")
            else:
                update_hand_data(hand_data)
        elif key.char == 'p':
            print("'p' key pressed, print hand data.")
            print_current_hand()
            
        elif key.char == 'd':
            print("'d' key pressed, discard tile.")
            # before discard
            hand_data0 = driver.execute_script("return GetHandData();")
            update_hand_data(hand_data0)
            print("Before discard:")
            print_current_hand()
            result = driver.execute_async_script("""
                // Selenium 的回調函數
                const done = arguments[arguments.length - 1];

                // 執行 discard 並處理 Promise
                discard(window.ownHand)
                    .then(tile => done(tile))        // 返回丟出的牌
                    .catch(err => done(err.message)); // 捕獲錯誤並返回錯誤訊息
            """)
            print(f"丟出的牌: {visualize_tile(filter_hand_data(result))}")
        elif key.char == 't':
            print("'t' key pressed, Toggle Auto Play.")
            if main_init is False:
                main_init = True
                auto_running = True
                driver.execute_script("main();")
            else:
                driver.execute_script("window.Auto_run = !window.Auto_run;")
                auto_running = not auto_running
    except AttributeError:
        pass

start_game()
inject_js()
with keyboard.Listener(on_press=on_press) as keyboard_listener:
    keyboard_listener.join()

Game started
JavaScript 已成功嵌入！
't' key pressed, Toggle Auto Play.


Unhandled exception in listener callback
Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pynput\_util\__init__.py", line 229, in inner
    return f(self, *args, **kwargs)
  File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pynput\keyboard\_win32.py", line 316, in _process
    self.on_press(key)
  File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pynput\_util\__init__.py", line 145, in inner
    if f(*args) is False:
  File "C:\Users\user\AppData\Local\Temp\ipykernel_6508\227345295.py", line 33, in on_press
    hand_data0 = driver.execute_script("return GetHandData();")
  File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 444, in execute_script
    return self.execute(command, {"script": script, "args": converted_args})["value"]
  File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\

'd' key pressed, discard tile.


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00007FF6025D6CB5+28821]
	(No symbol) [0x00007FF602543840]
	(No symbol) [0x00007FF6023E578A]
	(No symbol) [0x00007FF6023BF4F5]
	(No symbol) [0x00007FF602466247]
	(No symbol) [0x00007FF60247ECE2]
	(No symbol) [0x00007FF60245F0A3]
	(No symbol) [0x00007FF60242A778]
	(No symbol) [0x00007FF60242B8E1]
	GetHandleVerifier [0x00007FF60290FCAD+3408013]
	GetHandleVerifier [0x00007FF60292741F+3504127]
	GetHandleVerifier [0x00007FF60291B5FD+3455453]
	GetHandleVerifier [0x00007FF60269BDBB+835995]
	(No symbol) [0x00007FF60254EB5F]
	(No symbol) [0x00007FF60254A814]
	(No symbol) [0x00007FF60254A9AD]
	(No symbol) [0x00007FF60253A199]
	BaseThreadInitThunk [0x00007FFAF826259D+29]
	RtlUserThreadStart [0x00007FFAF9D0AF38+40]
